# Configure Scan & System Health Displays Test

## Table of Contents
#### [1 - Global Configuration](#1-global-configuration)
#### [2 - Setup Test Equipment](#2-test-equipment-setup)
#### [3 - Configure Scan Tests](#3-configure-scan-tests)
#### [4 - Debug Telescope](#4-debug-telescope)
#### [5 - Dashboard Links](#5-dashboard-links)
#### [6 - System Health Displays Tests](#6-system-health-displays-tests)

## 1 Global Configuration

### 1.1 Import dependencies

In [1]:
import sys

sys.path.append("../../src")

import json
import logging
import os
import pathlib
import time
from typing import List

import ska_ser_logging
from bokeh.io import output_notebook
from ska_oso_pdm.entities.common.target import (
    CrossScanParameters,
    FivePointParameters,
    RasterParameters,
    SinglePointParameters,
    StarRasterParameters,
)
from ska_oso_pdm.entities.sdp import BeamMapping
from ska_oso_scripting import oda_helper
from ska_oso_scripting.functions.devicecontrol.resource_control import get_request_json
from ska_oso_scripting.objects import SubArray, Telescope
from ska_tmc_cdm.messages.central_node.assign_resources import AssignResourcesRequest
from ska_tmc_cdm.messages.central_node.sdp import Channel
from ska_tmc_cdm.messages.subarray_node.configure import ConfigureRequest
from ska_tmc_cdm.messages.subarray_node.configure.core import ReceiverBand

from ska_mid_jupyter_notebooks.cluster.cluster import Environment, TangoDeployment
from ska_mid_jupyter_notebooks.dish.dish import TangoDishDeployment
from ska_mid_jupyter_notebooks.helpers.path import project_root
from ska_mid_jupyter_notebooks.obsconfig.config import ObservationSB
from ska_mid_jupyter_notebooks.obsconfig.target_spec import TargetSpec, get_default_target_specs_sb
from ska_mid_jupyter_notebooks.sut.rendering import TelescopeMononitorPlot
from ska_mid_jupyter_notebooks.sut.state import TelescopeDeviceModel, get_telescope_state
from ska_mid_jupyter_notebooks.sut.sut import TangoSUTDeployment, disable_qa
from ska_mid_jupyter_notebooks.test_equipment.rendering import get_test_equipment_monitor_plot
from ska_mid_jupyter_notebooks.test_equipment.state import get_equipment_model
from ska_mid_jupyter_notebooks.test_equipment.test_equipment import TangoTestEquipment

### 1.2 Setup Global Variables and Configuration

In [2]:
debug_mode = True  # This setting enables printing of diagnostics
enable_logging = True  # This enables logging and sets the global log_level to debug
dishlmc_enabled = True  # Set this to true if you have a dish LMC deployment
executon_environment = Environment.CI
branch_name = "at-1952-comm"  # Set this if you are using an on-demand deployment (i.e. Environment.CI)
if enable_logging:
    ska_ser_logging.configure_logging(logging.DEBUG)
test_equipment = TangoTestEquipment()
print(f"Test Equipment Configured: {test_equipment}")
# namespace_override parameter can be used to override auto-configured SUT namespace
sut_namespace_override = ""
subarray_count = 1
subarray_id = 1
sut = TangoSUTDeployment(
    branch_name,
    executon_environment,
    namespace_override=sut_namespace_override,
    subarray_index=subarray_id,
)
print(f"SUT configured: {str(sut)}")
dish_ids = ["001", "036"]
# namespace_override parameter can be used to override auto-configured dish namespace
dish_namespace_overrides = ["", ""]
dish_deployments: List[TangoDishDeployment] = []
if dishlmc_enabled:
    for i, d in enumerate(dish_ids):
        dish = TangoDishDeployment(
            f"ska{d}",
            branch_name=branch_name,
            environment=executon_environment,
            namespace_override=dish_namespace_overrides[i],
        )
        print(f"Dish {d} configured: {dish}")
        dish_deployments.append(dish)

timestr = time.strftime("%Y%m%d-%H%M")
notebook_output_dir = pathlib.Path(
    project_root(), f"notebook-execution-data/configure_scan_for_commissioning/execution-{timestr}"
)
os.makedirs(notebook_output_dir, exist_ok=True)
# we disable qa as it is not been properly verified
disable_qa()

Test Equipment Configured: TangoTestEquipment{namespace=test-equipment; tango_host=tango-databaseds.test-equipment.svc.miditf.internal.skao.int:10000; cluster_domain=miditf.internal.skao.int; cia_url=http://config-inspector.test-equipment.svc.miditf.internal.skao.int:8765}
SUT configured: TangoSUTDeployment{subarray_index=1; namespace=ci-ska-mid-itf-at-1952-comm; tango_host=tango-databaseds.ci-ska-mid-itf-at-1952-comm.svc.miditf.internal.skao.int:10000; cluster_domain=miditf.internal.skao.int; cia_url=http://config-inspector.ci-ska-mid-itf-at-1952-comm.svc.miditf.internal.skao.int:8765}
Dish 001 configured: TangoDishDeployment{dish_id=ska001; namespace=ci-dish-lmc-ska001-at-1952-comm; tango_host=tango-databaseds.ci-dish-lmc-ska001-at-1952-comm.svc.miditf.internal.skao.int:10000; cluster_domain=miditf.internal.skao.int; cia_url=http://config-inspector.ci-dish-lmc-ska001-at-1952-comm.svc.miditf.internal.skao.int:8765}
Dish 036 configured: TangoDishDeployment{dish_id=ska036; namespace=ci-

### 1.3 Test Connections to Namespaces

In [ ]:
sut.smoke_test()
test_equipment.smoke_test()
for dish_deployment in dish_deployments:
    dish_deployment.smoke_test()

### 1.4 Export System Configuration

In [3]:
deployment: TangoDeployment
for deployment in [sut, test_equipment, *dish_deployments]:
    deployment.export_chart_configuration(output_dir=notebook_output_dir)

Exporting configuration using http://config-inspector.ci-ska-mid-itf-at-1952-comm.svc.miditf.internal.skao.int:8765
ReleaseResponse (ci-ska-mid-itf-at-1952-comm): {
    "chart": "ska-mid-itf",
    "version": "22.0.1",
    "sub_charts": [
        {
            "chart": "ska-tango-base",
            "version": "0.4.9"
        },
        {
            "chart": "ska-tango-util",
            "version": "0.4.10"
        },
        {
            "chart": "ska-tmc-mid",
            "version": "0.19.1"
        },
        {
            "chart": "ska-csp-lmc-mid",
            "version": "0.18.2"
        },
        {
            "chart": "ska-mid-cbf-mcs",
            "version": "0.14.1"
        },
        {
            "chart": "ska-sdp",
            "version": "0.19.1"
        },
        {
            "chart": "ska-ser-config-inspector",
            "version": "0.2.2"
        },
        {
            "chart": "ska-tango-taranta",
            "version": "2.8.3"
        },
        {
            "c

## 2 Test Equipment Setup

Use the noise source at nominal levels for input to at least one SPFRx.  

### 2.1 Configure Test Equipment State 

In [4]:
test_equipment_state = get_equipment_model(test_equipment)
test_equipment.devices

['mid-itf/progattenuator/1',
 'mid-itf/siggen/1',
 'mid-itf/skysimctl/4',
 'mid-itf/spectana/1',
 'mid-sdp/queueconnector/01',
 'mid-sdp/queueconnector/02',
 'mid-sdp/queueconnector/03']

### 2.2 Print Test Equipment Diagnostics

In [ ]:
test_equipment.print_diagnostics()

### 2.3 Create Test Equipment Plot

In [6]:
monitor_plot = get_test_equipment_monitor_plot()
test_equipment_state.subscribe_to_test_equipment_state(monitor_plot.handle_device_state_change)
output_notebook()
monitor_plot.show()
test_equipment_state.activate()

Loading BokehJS ...

### 2.4 Turn offline Test Equipment devices ONLINE

In [ ]:
# set any offline devices to online
test_equipment.turn_online()

### 2.5 Display Test Equipment Device States 

In [ ]:
test_equipment_state.state["devices_states"]

### 2.6 Configure Signal Generator and set noise

In [7]:
frequency_to_set = 800e6

signal_generator = test_equipment.signal_generator
print(f"Current signal generator frequency: {signal_generator.frequency}")
signal_generator.write_attribute("frequency", frequency_to_set)
time.sleep(1)
print(f"Updated signal generator frequency: {signal_generator.frequency}")
assert (
    signal_generator.frequency == frequency_to_set
), f"Frequency required is {frequency_to_set} but got {signal_generator.frequency}"

Current signal generator frequency: 800000000.0
Updated signal generator frequency: 800000000.0


## 3 Configure Scan Tests

Use OSO scripting interface to TMC to run the script for assigning resources for a single sub-array, configuring and running a  scan.  

### 3.1 Setup Telescope Monitoring

#### 3.1.1 Configure Telescope Monitoring

In [8]:
# setup monitoring
# use telescope state object for state monitoring
device_model = TelescopeDeviceModel(dish_ids, subarray_count)
telescope_state = get_telescope_state(device_model, sut)
# use monitor plot as a dashboard
telescope_monitor_plot = TelescopeMononitorPlot(plot_width=900, plot_height=200)
# set up events to monitor
telescope_state.subscribe_to_on_off(telescope_monitor_plot.observe_telescope_on_off)
telescope_state.subscribe_to_subarray_resource_state(
    telescope_monitor_plot.observe_subarray_resources_state
)
telescope_state.subscribe_to_subarray_configurational_state(
    telescope_monitor_plot.observe_subarray_configuration_state
)
telescope_state.subscribe_to_subarray_scanning_state(
    telescope_monitor_plot.observe_subarray_scanning_state
)
output_notebook()

Loading BokehJS ...

#### 3.1.2 Open the inline dashboard
Start the simple inline dashboard showing current state of the Telescope and resource assignment and configuration status.

In [10]:
telescope_monitor_plot.show()
telescope_state.activate()
telescope_state.wait_til_ready(2)

### 3.2 Print System Diagnostics

#### 3.2.1 Print TMC Diagnostics

In [ ]:
sut.print_tmc_diagnostics()

#### 3.2.2 Print CSP-LMC Diagnostics

In [ ]:
sut.print_csp_diagnostics()

#### 3.2.3 Print CBF Diagnostics

In [ ]:
sut.print_cbf_diagnostics()

#### 3.2.4 Print SDP Diagnostics

In [ ]:
sut.print_sdp_diagnostics()

#### 3.2.5 Print Dish-LMC Diagnostics

In [11]:
for dish_deployment in dish_deployments:
    print(f"Dish {dish_deployment.dish_id} - {dish_deployment.namespace}: Diagnostics")

    dish_deployment.print_diagnostics()

Dish ska001 - ci-dish-lmc-ska001-at-1952-comm: Diagnostics
ska001: ComponentStates: "{'SPF': {'operatingmode': <SPFOperatingMode.STANDBY_LP: 2>, 'powerstate': <SPFPowerState.LOW_POWER: 1>, 'healthstate': <HealthState.UNKNOWN: 3>, 'bandinfocus': <BandInFocus.UNKNOWN: 0>, 'b1capabilitystate': <SPFCapabilityStates.STANDBY: 1>, 'b2capabilitystate': <SPFCapabilityStates.STANDBY: 1>, 'b3capabilitystate': <SPFCapabilityStates.STANDBY: 1>, 'b4capabilitystate': <SPFCapabilityStates.STANDBY: 1>, 'b5acapabilitystate': <SPFCapabilityStates.STANDBY: 1>, 'b5bcapabilitystate': <SPFCapabilityStates.STANDBY: 1>}, 'DS': {'healthstate': <HealthState.UNKNOWN: 3>, 'operatingmode': <DSOperatingMode.STANDBY_LP: 2>, 'pointingstate': <PointingState.READY: 0>, 'achievedtargetlock': None, 'indexerposition': <IndexerPosition.B2: 2>, 'powerstate': <DSPowerState.LOW_POWER: 3>, 'desiredpointingaz': [1715757700.0, 0.0], 'desiredpointingel': [1715757700.0, 0.0], 'achievedpointing': [1715757700.0, 0.0, 85.0], 'achieved

#### 3.2.6 Print Full System Diagnostics

In [ ]:
print("SUT: Full Diagnostics")
sut.print_full_diagnostics()

for dish_deployment in dish_deployments:
    print(f"Dish {dish_deployment.dish_id}: Full Diagnostics")
    dish_deployment.print_full_diagnostics()

print("Test Equipment: Full Diagnostics")
test_equipment.print_full_diagnostics()

### 3.3 Setup ODA

In [12]:
os.environ["ODA_URI"] = (
    "http://ingress-nginx-controller-lb-default.ingress-nginx.svc.miditf.internal.skao.int/ska-db-oda/api/v1/"
)
eb_id = oda_helper.create_eb()
print(f"Execution Block ID: {eb_id}")

Execution Block ID: eb-miditf-20240515-00015


### 3.4 Initialise Telescope and Subarray
Create Subarray and Telescope instances.

In [13]:
sub = SubArray(subarray_id)
tel = Telescope()

### 3.5 Load VCC Configuration in TMC

In [15]:
# This should only be executed for a fresh deployment (i.e. Telescope is OFF.
# If you have restarted the subarray, you should not run this command
sut.load_dish_vcc_config()

CSP Controller: adminMode=1; State=DISABLE
CSP Controller: adminMode=0; State=DISABLE after 0s.
CSP Controller: adminMode=0; State=DISABLE after 1s.
CSP Controller: adminMode=0; State=OFF after 3s.
CSP Controller: adminMode=0; State=OFF
TMC Central Node isDishVccConfigSet=False after 0s
TMC Central Node isDishVccConfigSet=False after 1s
TMC Central Node isDishVccConfigSet=False after 3s
TMC Central Node isDishVccConfigSet=False after 7s
TMC Central Node isDishVccConfigSet=True after 15s
CSP Controller: adminMode=0; State=OFF
TMC Central Node: isDishVccConfigSet=True; dishvccvalidationstatus={"dish": "ALL DISH OK", "ska_mid/tm_leaf_node/csp_master": "TMC and CSP Master Dish Vcc Version is Same"}
TMC CSP Master Leaf Node: sourceDishVccConfig={"interface": "https://schema.skao.int/ska-mid-cbf-initsysparam/1.0", "tm_data_sources": ["car://gitlab.com/ska-telescope/ska-telmodel-data?main#tmdata"], "tm_data_filepath": "instrument/ska1_mid_psi/ska-mid-cbf-system-parameters.json"}; dishVccConfi

### 3.6 Turn telescope ON

In [16]:
# set to ON only if OFF
# If you have restarted the subarray, you should not run this command (Telescope is already ON)
# dish_lmc mode must be in LP_standby and before trying to turn the telescope ON
# Takes about 1m20s
if telescope_monitor_plot.on_off_state == "OFF":  # e.g. purple
    tel.on()
else:
    assert (
        telescope_monitor_plot.on_off_state == "ON"
    ), f"Cant continue with telescope in {telescope_monitor_plot.on_off_state}"

1|2024-05-15T07:48:06.612Z|DEBUG|MainThread|wrapper|oda_helper.py#244||Capturing the request function telescope_on in an ExecutionBlock
1|2024-05-15T07:48:06.615Z|INFO|MainThread|_call_and_wait_for_transition|common.py#147||Using pub/sub to track telescopeState of ska_mid/tm_central/central_node
1|2024-05-15T07:48:06.617Z|DEBUG|MainThread|subscribe_event|tango_executor.py#136||Observing ska_mid/tm_central/central_node/telescopeState
1|2024-05-15T07:48:06.621Z|INFO|MainThread|execute|tango_executor.py#243||Executing command: <Command('ska_mid/tm_central/central_node', 'TelescopeOn', )>
1|2024-05-15T07:48:06.646Z|INFO|MainThread|wait_for_transition|common.py#338||Waiting for telescopeState to transition to ON
1|2024-05-15T07:48:06.650Z|INFO|MainThread|read_event|tango_executor.py#184||timeout None to wait for read an event from queue
1|2024-05-15T07:48:07.547Z|DEBUG|Dummy-11|notify|tango_executor.py#204||Received event: EventData[
     attr_name = 'tango://tango-databaseds.ci-ska-mid-itf

### 3.7 Observation Definition

#### 3.7.1 Create the high level observation specifications in terms of target specs

Note :- Users may currently modify the values by replacing the example values as given for each field within Target specification section.

In [17]:
dish_ids = [d.dish_id.upper() for d in dish_deployments]
default_target_specs = get_default_target_specs_sb(dish_ids)
observation = ObservationSB(target_specs=default_target_specs)

target_specs = {
    "flux calibrator": TargetSpec(
        target_sb_detail={
            "co_ordinate_type": "Equatorial",
            "ra": "19:24:51.05 degrees",
            "dec": "-29:14:30.12 degrees",
            "reference_frame": "ICRS",
            "unit": ("hourangle", "deg"),
            "pointing_pattern_type": {
                "single_pointing_parameters": SinglePointParameters(
                    offset_x_arcsec=0.0, offset_y_arcsec=0.0
                ),
                "raster_parameters": RasterParameters(
                    row_length_arcsec=0.0,
                    row_offset_arcsec=0.0,
                    n_rows=1,
                    pa=0.0,
                    unidirectional=False,
                ),
                "star_raster_parameters": StarRasterParameters(
                    row_length_arcsec=0.0,
                    n_rows=1,
                    row_offset_angle=0.0,
                    unidirectional=False,
                ),
                "five_point_parameters": FivePointParameters(offset_arcsec=0.0),
                "cross_scan_parameters": CrossScanParameters(offset_arcsec=0.0),
                "active_pointing_pattern_type": "single_pointing_parameters",
            },
        },
        scan_type="flux calibrator",
        band=ReceiverBand.BAND_2,
        channelisation="vis_channels9",
        polarisation="all",
        processing="test-receive-addresses",
        dish_ids=dish_ids,
        target=None,
    ),
    "M87": TargetSpec(
        target_sb_detail={
            "co_ordinate_type": "Equatorial",
            "ra": "19:24:51.05 degrees",
            "dec": "-29:14:30.12 degrees",
            "reference_frame": "ICRS",
            "unit": ("hourangle", "deg"),
            "pointing_pattern_type": {
                "single_pointing_parameters": SinglePointParameters(
                    offset_x_arcsec=0.0, offset_y_arcsec=0.0
                ),
                "raster_parameters": RasterParameters(
                    row_length_arcsec=0.0,
                    row_offset_arcsec=0.0,
                    n_rows=1,
                    pa=0.0,
                    unidirectional=False,
                ),
                "star_raster_parameters": StarRasterParameters(
                    row_length_arcsec=0.0,
                    n_rows=1,
                    row_offset_angle=0.0,
                    unidirectional=False,
                ),
                "five_point_parameters": FivePointParameters(offset_arcsec=0.0),
                "cross_scan_parameters": CrossScanParameters(offset_arcsec=0.0),
                "active_pointing_pattern_type": "single_pointing_parameters",
            },
        },
        scan_type="M87",
        band=ReceiverBand.BAND_2,
        channelisation="vis_channels10",
        polarisation="all",
        processing="test-receive-addresses",
        dish_ids=dish_ids,
        target=None,
    ),
}


channel_configuration = [
    Channel(
        spectral_window_id="fsp_1_channels",
        count=14880,
        start=0,
        stride=2,
        freq_min=0.35e9,
        freq_max=0.368e9,
        link_map=[[0, 0], [200, 1], [744, 2], [944, 3]],
    )
]

for key, value in target_specs.items():
    observation.add_channel_configuration(value.channelisation, channel_configuration)

observation.add_target_specs(target_specs)

for target_id, target in target_specs.items():
    observation.add_scan_type_configuration(
        config_name=target_id,
        beams={"vis0": BeamMapping(beam_id="vis0", field_id="M83")},
        derive_from=".default",
    )
scan_def_id = "flux calibrator"
observation.add_scan_sequence([scan_def_id])

#### 3.7.2 Mid configuration schema input used by observing commands

[Configuration Schemas-OET→TMC(Mid)](https://developer.skao.int/projects/ska-telmodel/en/latest/)

In [18]:
telescope_monitor_plot.show()

#### 3.7.3 Create Scheduling Block Definition(SBD) Instance and save it into the ODA

In [19]:
observation.eb_id = eb_id
pdm_allocation = observation.generate_pdm_object_for_sbd_save(target_specs)

sbd = oda_helper.save(pdm_allocation)
sbd_id = sbd.sbd_id
pdm_allocation.sbd_id = sbd_id
print(f"Saved Scheduling Block Definition Instance in ODA: SBD_ID={sbd_id}")

1|2024-05-15T07:48:25.084Z|DEBUG|MainThread|save|oda_helper.py#66||Sending request to http://ingress-nginx-controller-lb-default.ingress-nginx.svc.miditf.internal.skao.int/ska-db-oda/api/v1/sbds
Saved Scheduling Block Definition Instance in ODA: SBD_ID=sbd-miditf-20240515-00016


### 3.8 Assign Resources
Assign the requested resources to a Subarray

In [20]:
assign_request = observation.generate_allocate_config_sb(pdm_allocation).as_object

if debug_mode:
    request_json = get_request_json(assign_request, AssignResourcesRequest, True)
    print("AssignResourcesRequest:", json.dumps(json.loads(request_json), indent=2))

sub.assign_from_cdm(assign_request, timeout=120)

1|2024-05-15T07:48:28.867Z|INFO|MainThread|convert_channels|sdp.py#290||Setting ChannelConfiguration id:vis_channels9 , spectral_windows:[<SpectralWindow(spectral_window_id=fsp_1_channels, count=14880, start=0, stride=2, freq_min=350000000.0, freq_max=368000000.0, link_map=[(0, 0), (200, 1), (744, 2), (944, 3)])>] 
1|2024-05-15T07:48:28.870Z|INFO|MainThread|convert_spectral_window|sdp.py#269||Setting channel attribute -> count:14880 , start:0 , stride:2, freq_min:350000000.0, freq_max:368000000.0 , link_map:[(0, 0), (200, 1), (744, 2), (944, 3)] 
1|2024-05-15T07:48:28.874Z|INFO|MainThread|convert_channels|sdp.py#290||Setting ChannelConfiguration id:vis_channels , spectral_windows:[<SpectralWindow(spectral_window_id=fsp_1_channels, count=14880, start=0, stride=2, freq_min=350000000.0, freq_max=368000000.0, link_map=[(0, 0), (200, 1), (744, 2), (944, 3)])>] 
1|2024-05-15T07:48:28.879Z|INFO|MainThread|convert_spectral_window|sdp.py#269||Setting channel attribute -> count:14880 , start:0 ,

In [21]:
telescope_monitor_plot.show()

### 3.9 Configure Scan
Configure the telescope  on first target in sequence - may be modified to configure and run multiple targets at a later time.

In [22]:
configure_object = observation.generate_scan_config_sb(
    pdm_observation_request=pdm_allocation,
    scan_definition_id=scan_def_id,
    scan_duration=10.0,
).as_object

if debug_mode:
    cfg_json = get_request_json(configure_object, ConfigureRequest)
    print(f"ConfigureRequest={cfg_json}")

sub.configure_from_cdm(configure_object, timeout=120)
time.sleep(2)

1|2024-05-15T07:49:02.357Z|DEBUG|MainThread|_open_nexus|backend.py#766||Querying Nexus: GET https://artefact.skao.int/repository/raw-telmodel/gitlab.com/ska-telescope/ska-telmodel/tmdata/1.13.0
ConfigureRequest={"interface": "https://schema.skao.int/ska-tmc-configure/2.2", "transaction_id": "txn-....-00001", "pointing": {"target": {"ra": "01:17:39.40333333", "dec": "-29:14:30.12", "reference_frame": "ICRS", "target_name": "flux calibrator"}}, "dish": {"receiver_band": "2"}, "sdp": {"interface": "https://schema.skao.int/ska-sdp-configure/0.4", "scan_type": "flux calibrator"}, "csp": {"interface": "https://schema.skao.int/ska-csp-configure/2.0", "subarray": {"subarray_name": "dummy name"}, "common": {"config_id": "sbi-mvp01-20200325-00001-science_A", "frequency_band": "2", "subarray_id": 1}, "cbf": {"fsp": [{"fsp_id": 1, "function_mode": "CORR", "frequency_slice_id": 1, "zoom_factor": 0, "integration_factor": 10, "output_link_map": [[0, 0], [200, 1]], "channel_offset": 0}, {"fsp_id": 2, 

In [23]:
telescope_monitor_plot.show()

### 3.10 Post Observation teardown
If the observation executed successfully, you can use the following commands to reset the telescope.

#### 3.10.1 Clear scan configuration 

In [24]:
sub.end()
telescope_monitor_plot.show()

1|2024-05-15T07:51:05.041Z|DEBUG|MainThread|wrapper|oda_helper.py#244||Capturing the request function end in an ExecutionBlock
1|2024-05-15T07:51:05.045Z|INFO|MainThread|_call_and_wait_for_transition|common.py#147||Using pub/sub to track obsState of ska_mid/tm_subarray_node/1
1|2024-05-15T07:51:05.047Z|DEBUG|MainThread|subscribe_event|tango_executor.py#136||Observing ska_mid/tm_subarray_node/1/obsState
1|2024-05-15T07:51:05.049Z|INFO|MainThread|execute|tango_executor.py#243||Executing command: <Command('ska_mid/tm_subarray_node/1', 'End', )>
1|2024-05-15T07:51:05.069Z|INFO|MainThread|wait_for_transition|common.py#338||Waiting for obsState to transition to ObsState.IDLE
1|2024-05-15T07:51:05.072Z|INFO|MainThread|read_event|tango_executor.py#184||timeout None to wait for read an event from queue
1|2024-05-15T07:51:06.800Z|DEBUG|Dummy-11|notify|tango_executor.py#204||Received event: EventData[
     attr_name = 'tango://tango-databaseds.ci-ska-mid-itf-at-1952-comm.svc.miditf.internal.skao.

#### 3.10.2 Release Subarray resources

In [25]:
sub.release()
telescope_monitor_plot.show()

1|2024-05-15T07:51:12.560Z|DEBUG|MainThread|wrapper|oda_helper.py#244||Capturing the request function release_all_resources in an ExecutionBlock
1|2024-05-15T07:51:12.567Z|INFO|MainThread|_call_and_wait_for_transition|common.py#147||Using pub/sub to track obsState of ska_mid/tm_subarray_node/1
1|2024-05-15T07:51:12.569Z|DEBUG|MainThread|subscribe_event|tango_executor.py#136||Observing ska_mid/tm_subarray_node/1/obsState
1|2024-05-15T07:51:12.571Z|INFO|MainThread|execute|tango_executor.py#243||Executing command: <Command('ska_mid/tm_central/central_node', 'ReleaseResources', '{"interface": "https://schema.skao.int/ska-tmc-releaseresources/2.1", "subarray_id": 1, "release_all": true}')>
1|2024-05-15T07:51:12.604Z|INFO|MainThread|wait_for_transition|common.py#338||Waiting for obsState to transition to ObsState.RESOURCING
1|2024-05-15T07:51:12.606Z|INFO|MainThread|read_event|tango_executor.py#184||timeout 20 to wait for read an event from queue
1|2024-05-15T07:51:12.611Z|DEBUG|Dummy-11|not

### 3.11 Reset the Telescope (On Failure)
Set booleans to True to reset the system after a failed execution.

#### 3.11.1 Reset the Subarray

In [27]:
sub.abort()
time.sleep(3)
sub.restart()

1|2024-05-15T07:51:31.085Z|DEBUG|MainThread|wrapper|oda_helper.py#244||Capturing the request function abort in an ExecutionBlock
1|2024-05-15T07:51:31.088Z|INFO|MainThread|_call_and_wait_for_transition|common.py#147||Using pub/sub to track obsState of ska_mid/tm_subarray_node/1
1|2024-05-15T07:51:31.089Z|DEBUG|MainThread|subscribe_event|tango_executor.py#136||Observing ska_mid/tm_subarray_node/1/obsState
1|2024-05-15T07:51:31.090Z|INFO|MainThread|execute|tango_executor.py#243||Executing command: <Command('ska_mid/tm_subarray_node/1', 'Abort', )>
1|2024-05-15T07:51:31.100Z|DEBUG|MainThread|unsubscribe_event|tango_executor.py#154||Unobserving ska_mid/tm_subarray_node/1/obsState
1|2024-05-15T07:51:31.102Z|DEBUG|MainThread|send_request_response|oda_helper.py#218||Sending request to http://ingress-nginx-controller-lb-default.ingress-nginx.svc.miditf.internal.skao.int/ska-db-oda/api/v1/ebs/eb-miditf-20240515-00015/request_response with body {'error': {'detail': 'DevFailed(args = (DevError(de

DevFailed: DevFailed[
DevError[
    desc = ska_tango_base.faults.StateModelError: Abort command not permitted in observation state EMPTY
           
  origin = Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tango/device_server.py", line 85, in wrapper
    return get_worker().execute(fn, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/tango/green.py", line 101, in execute
    return fn(*args, **kwargs)
  File "/app/src/ska_tmc_subarraynode/subarray_node.py", line 546, in is_Abort_allowed
    super().is_Abort_allowed()
  File "/usr/local/lib/python3.10/dist-packages/ska_tango_base/subarray/subarray_device.py", line 866, in is_Abort_allowed
    raise StateModelError(
ska_tango_base.faults.StateModelError: Abort command not permitted in observation state EMPTY

  reason = PyDs_PythonError
severity = ERR]

DevError[
    desc = Failed to execute command_inout on device ska_mid/tm_subarray_node/1, command Abort
  origin = virtual Tango::Dev

#### 3.11.2 Reset the Dish

In [28]:
dish_deployment: TangoDishDeployment
for dish_deployment in dish_deployments:
    dish_deployment.reset_dish()

ska001: aborting dish operation


## 4 Debug Telescope

### 4.1 Display Telescope State

In [ ]:
telescope_state.state

### 4.2 Display Telescope State Subscriptions

In [ ]:
telescope_state.state_monitor.subscriptions

### 4.3 Display Dish LMC State

In [ ]:
dishes_to_debug = [d.dish_id for d in dish_deployments]
for dish in dish_deployments:
    if dish.dish_id in dishes_to_debug:
        dish.print_diagnostics()

### 4.4 Display CSP LMC State

In [ ]:
sut.print_csp_diagnostics()

### 4.5 Display TMC State

In [ ]:
sut.print_tmc_diagnostics()

### 4.6 Display SDP State

In [ ]:
sut.print_sdp_diagnostics()

## 5 Dashboard Links

### 5.1 Taranta

In [ ]:
print(f"SUT: {sut.taranta_endpoint}")
print(f"Test Equipment: {test_equipment.taranta_endpoint}")
for dish in dish_deployments:
    print(f"Dish LMC {dish.dish_id}: {dish.taranta_endpoint}")

### 5.2 Data Product Dashboard

https://k8s.miditf.internal.skao.int/ska-dpd/dashboard/

### 5.3 SDP Signal Displays

In [ ]:
print(sut.signal_displays_endpoint)

## 6 Configure System for Scan Commissioning Test

### Copied from Jama 2024-04-15

### 6.1 Bring the sub-systems to STANDBY mode using Telescope ON script or command from TMC as applicable.

Running steps 1-3.6 should get the telescope into a standby state which can be checked by running 3.2.2 - 3.2.6

### 6.2 Inspect the TMC API or dashboard to view the indication of connectivity status and mode of operation of the sub-systems.

### 6.3 Change the telescope state to operational
Running steps 3.3. - 3.9 should get the telescope to operational mode which can be checked by running steps 3.2.2 - 3.2.6